In [4]:
import pandas as pd
from tabular_data import load_airbnb
from tabular_data import clean_tabular_data

from sklearn import model_selection
from sklearn import linear_model
from sklearn import datasets
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.linear_model import SGDRegressor

In [3]:
path = '/Users/ryanhughes/Desktop/Aicore/Airbnb/Airbnb/AirbnbData/Raw_Data/tabular_data/listing.csv'
Label = "Price_Night"

In [51]:
class MGS:

    def __init__(self):
        self.train_split ={}

    def sgd(self, path, label):
        Airbnb_data = pd.read_csv(path)

        (X,y) = load_airbnb(Airbnb_data, label)

        X = scale(X)
        y = scale(y)
        xtrain, xtest, ytrain, ytest = model_selection.train_test_split(X, y, test_size=0.3)

        xvalid, xtest, yvalid, ytest = model_selection.train_test_split(xtest, ytest, test_size=0.5)

        self.train_split = {
            'xtrain' : xtrain,
            'xtest' : xtest,
            'ytrain' : ytrain,
            'ytest' : ytest,
            'xvalid' :xvalid,
            'yvalid' : yvalid
        }

In [52]:
mgs = MGS()
mgs.sgd('/Users/ryanhughes/Desktop/Aicore/Airbnb/Airbnb/AirbnbData/Raw_Data/tabular_data/listing.csv', 'Price_Night')
model_dict = mgs.train_split

In [53]:
sgdr = SGDRegressor()
print(sgdr)

SGDRegressor()


In [54]:
model_dict["xtrain"]

array([[ 0.24661594,  0.06167277,  0.03246415, ...,  0.19711765,
         1.30722058,         nan],
       [ 0.24661594,  0.06167277,  0.12205793, ...,  0.19711765,
        -0.25594374,         nan],
       [ 1.29404095,  0.60932699,  0.13620432, ..., -0.35283441,
         1.23925691,         nan],
       ...,
       [-0.80080907, -0.48598145,  0.34840013, ..., -0.35283441,
        -0.1200164 ,         nan],
       [-0.80080907, -0.48598145, -0.45322849, ...,  0.7470697 ,
        -1.00354406,         nan],
       [-0.27709656, -0.48598145, -0.1372925 , ..., -1.45273851,
         1.78296624,         nan]])

In [47]:
sgdr.fit(model_dict["xtrain"], model_dict["ytrain"])

score = sgdr.score(mgs.train_split["xtrain"], mgs.train_split["ytrain"])
print("R-squared:", score)

ValueError: Input X contains NaN.
SGDRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [44]:
df = pd.read_csv(path)
df = clean_tabular_data(df)
df = df.iloc[: ,:-1]
df_features = df.select_dtypes(['float64', 'int64'])
df_features= pd.DataFrame(df_features)

df_features

,beds,bathrooms,Price_Night,Cleanliness_rating,Accuracy_rating,Communication_rating,Location_rating,Check-in_rating,Value_rating,amenities_count
0,1.0,1.0,105,4.6,4.7,4.3,5.0,4.3,4.3,13.0
1,3.0,0.0,92,4.3,4.7,4.6,4.9,4.7,4.5,8.0
2,2.0,1.5,52,4.2,4.6,4.8,4.8,4.8,4.7,51.0
3,1.0,1.0,132,4.8,4.9,4.9,4.9,5.0,4.6,23.0
5,3.0,1.0,143,5.0,4.9,5.0,4.7,5.0,4.7,32.0
...,...,...,...,...,...,...,...,...,...,...
982,2.0,1.5,240,4.9,5.0,5.0,5.0,4.9,4.8,33.0
983,1.0,1.0,78,4.8,5.0,4.9,4.9,5.0,4.9,54.0
984,2.0,1.5,113,4.8,5.0,5.0,5.0,5.0,4.8,38.0
985,3.0,2.0,80,4.7,4.8,5.0,5.0,5.0,4.7,24.0


In [62]:
import ast

def remove_rows_with_missing_ratings(Dataframe):
    rating_columns=['Cleanliness_rating','Accuracy_rating','Communication_rating','Location_rating','Check-in_rating','Value_rating','Description']
    Dataframe = Dataframe.dropna(subset = rating_columns)
    return Dataframe

def clean_strings(str):
    try:
        clean_str = ast.literal_eval(str)
        clean_str.remove("About this space")
        clean_str.remove("What this place offers")
        clean_str = "".join(clean_str)
        return clean_str
    except Exception as e:
        return str

def clean_description_strings(dataframe):
    dataframe[["Description", "Amenities"]] = dataframe[["Description", "Amenities"]].apply(clean_strings)
    dataframe["Description"].replace([r"\\n", "\n", r"\'"], [" "," ",""], regex=True, inplace=True)
    return dataframe

def set_default_feature_values(dataframe):
    columns = ["beds", "bathrooms", "bedrooms", "guests"]
    dataframe[columns] = dataframe[columns].fillna(1)
    return dataframe

In [24]:
def clean_tabular_data(dataframe):

    #dataframe = dataframe.iloc[: ,:-1]
    dataframe = remove_rows_with_missing_ratings(dataframe)
    dataframe = clean_description_strings(dataframe)
    dataframe = set_default_feature_values(dataframe)
    return dataframe

In [ ]:
df = pd.read_csv(path)
df1 = remove_rows_with_missing_ratings(df)
df2 = clean_description_strings(df1)
df.head()


In [ ]:
df2.head()

In [74]:
def Load_airbnb(dataframe, label):
    df_features = clean_tabular_data(dataframe)
    df_features = dataframe.select_dtypes(['float64', 'int64'])
    features = df_features.to_numpy()
    labels = df_features[label]
    labels = labels.to_numpy()
    df_features.drop([label],axis=1, inplace=True)
    return (features,labels)

In [84]:
air_df = pd.read_csv(path)
air_df[["beds", "bathrooms", "bedrooms", "guests"]] = air_df[["beds", "bathrooms", "bedrooms", "guests"]].fillna(1)
#air_df[['Cleanliness_rating','Accuracy_rating','Communication_rating','Location_rating','Check-in_rating','Value_rating','Description']] = air_df[['Cleanliness_rating','Accuracy_rating','Communication_rating','Location_rating','Check-in_rating','Value_rating','Description']].dropna()
air_df.dropna(inplace = True, subset = ['Cleanliness_rating','Accuracy_rating','Communication_rating','Location_rating','Check-in_rating','Value_rating','Description'])
air_df

,ID,Category,Title,Description,Amenities,Location,guests,beds,bathrooms,Price_Night,Cleanliness_rating,Accuracy_rating,Communication_rating,Location_rating,Check-in_rating,Value_rating,amenities_count,url,bedrooms,Unnamed: 19
0,f9dcbd09-32ac-41d9-a0b1-fdb2793378cf,Treehouses,Red Kite Tree Tent - Ynys Affalon,"['About this space', ""Escape to one of these t...","['What this place offers', 'Bathroom', 'Shampo...",Llandrindod Wells United Kingdom,2,1.0,1.0,105,4.6,4.7,4.3,5.0,4.3,4.3,13.0,https://www.airbnb.co.uk/rooms/26620994?adults...,1,NaN
1,1b4736a7-e73e-45bc-a9b5-d3e7fcf652fd,Treehouses,Az Alom Cabin - Treehouse Tree to Nature Cabin,"['About this space', ""Come and spend a romanti...","['What this place offers', 'Bedroom and laundr...",Guyonvelle Grand Est France,3,3.0,0.0,92,4.3,4.7,4.6,4.9,4.7,4.5,8.0,https://www.airbnb.co.uk/rooms/27055498?adults...,1,NaN
2,d577bc30-2222-4bef-a35e-a9825642aec4,Treehouses,Cabane Entre Les Pins\n🌲🏕️🌲,"['About this space', 'Rustic cabin between the...","['What this place offers', 'Scenic views', 'Ga...",Duclair Normandie France,4,2.0,1.5,52,4.2,4.6,4.8,4.8,4.8,4.7,51.0,https://www.airbnb.co.uk/rooms/51427108?adults...,1,NaN
3,ca9cbfd4-7798-4e8d-8c17-d5a64fba0abc,Treehouses,Tree Top Cabin with log burner & private hot tub,"['About this space', 'The Tree top cabin is si...","['What this place offers', 'Bathroom', 'Hot wa...",Barmouth Wales United Kingdom,2,1.0,1.0,132,4.8,4.9,4.9,4.9,5.0,4.6,23.0,https://www.airbnb.co.uk/rooms/49543851?adults...,1,NaN
5,cfe479b9-c8f8-44af-9bc6-46ede9f14bb5,Treehouses,Treehouse near Paris Disney,"['About this space', 'Charming cabin nestled i...","['What this place offers', 'Bathroom', 'Hair d...",Le Plessis-Feu-Aussoux Île-de-France France,4,3.0,1.0,143,5.0,4.9,5.0,4.7,5.0,4.7,32.0,https://www.airbnb.co.uk/rooms/935398?adults=1...,2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
982,cfbc88da-e88c-415a-b397-108d0948c4ba,Beachfront,Lancing Beach Apartment,"['About this space', 'An apartment directly on...","['What this place offers', 'Bathroom', 'Hair d...",Lancing United Kingdom,4,2.0,1.5,240,4.9,5.0,5.0,5.0,4.9,4.8,33.0,https://www.airbnb.co.uk/rooms/12680472?adults...,2,NaN
983,4fea5054-f999-4c07-addc-67e4d893deab,Beachfront,Apartment,"['About this space', 'Light roomy space with o...","['What this place offers', 'Bathroom', 'Hair d...",Brighton and Hove England United Kingdom,2,1.0,1.0,78,4.8,5.0,4.9,4.9,5.0,4.9,54.0,https://www.airbnb.co.uk/rooms/48565992?adults...,1,NaN
984,282118e2-049e-4d9f-b2f2-b47477881b07,Beachfront,Sea front flat with a stunning view!,"['About this space', 'This specious two bedroo...","['What this place offers', 'Scenic views', 'Be...",East Sussex England United Kingdom,4,2.0,1.5,113,4.8,5.0,5.0,5.0,5.0,4.8,38.0,https://www.airbnb.co.uk/rooms/49742544?adults...,2,NaN
985,9ebf9cec-624e-480e-8704-dffa7cb1fe51,Beachfront,MP713 - Camber Sands Holiday Park - Sleeps 6 +...,"['About this space', 'With all the modern amen...","['What this place offers', 'Bathroom', 'Hot wa...",Camber England United Kingdom,6,3.0,2.0,80,4.7,4.8,5.0,5.0,5.0,4.7,24.0,https://www.airbnb.co.uk/rooms/47777462?adults...,2,NaN


In [76]:
X,y = Load_airbnb(air_df, Label)
#print(y)

/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_55585/1383926063.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe[["Description", "Amenities"]] = dataframe[["Description", "Amenities"]].apply(clean_strings)
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_55585/1383926063.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataframe["Description"].replace([r"\\n", "\n", r"\'"], [" "," ",""], regex=True, inplace=True)
/var/folders/k0/hpk5zvqn2rdfdj_f5sv8ptp80000gn/T/ipykernel_55585/1383926063.py:25: SettingWithCopyWarning: 
A va

In [77]:
xtrain, xtest, ytrain, ytest = model_selection.train_test_split(X, y, test_size=0.3)

In [80]:
xvalid, xtest, yvalid, ytest = model_selection.train_test_split(xtest, ytest, test_size=0.5)

In [78]:
sgdr = SGDRegressor()
print("sgdr:")
print(sgdr)

sgdr:
SGDRegressor()


In [79]:
sgdr.fit(xtrain,ytrain)

ValueError: Input X contains NaN.
SGDRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [24]:
import pandas as pd
Airbnb_data = pd.read_csv('/Users/ryanhughes/Desktop/Aicore/Airbnb/Airbnb/AirbnbData/Raw_Data/tabular_data/listing.csv')

In [31]:
Airbnb_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 829 entries, 0 to 829
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Unnamed: 0            829 non-null    int64  
 1   ID                    829 non-null    object 
 2   Category              829 non-null    object 
 3   Title                 829 non-null    object 
 4   Description           829 non-null    object 
 5   Amenities             829 non-null    object 
 6   Location              829 non-null    object 
 7   guests                829 non-null    int64  
 8   beds                  829 non-null    float64
 9   bathrooms             829 non-null    float64
 10  Price_Night           829 non-null    int64  
 11  Cleanliness_rating    829 non-null    float64
 12  Accuracy_rating       829 non-null    float64
 13  Communication_rating  829 non-null    float64
 14  Location_rating       829 non-null    float64
 15  Check-in_rating       829 no

In [27]:
string_values_bedrooms = Airbnb_data.loc[Airbnb_data['bedrooms'].apply(lambda x: isinstance(x, str)), 'bedrooms']
string_values_guests = Airbnb_data.loc[Airbnb_data['guests'].apply(lambda x: isinstance(x, str)), 'guests']

In [29]:
Airbnb_data = Airbnb_data[pd.to_numeric(Airbnb_data['guests'], errors='coerce').notnull()]
Airbnb_data = Airbnb_data[pd.to_numeric(Airbnb_data['bedrooms'], errors='coerce').notnull()]

In [30]:
columns = ["guests", "bedrooms"]
for column in columns:
    Airbnb_data[column] = Airbnb_data[column].astype(int)

In [18]:
string_values_bedrooms

0      1
1      1
2      1
3      1
4      2
      ..
825    2
826    1
827    2
828    2
829    2
Name: bedrooms, Length: 830, dtype: object

In [19]:
string_values_guests

0      2
1      3
2      4
3      2
4      4
      ..
825    4
826    2
827    4
828    6
829    4
Name: guests, Length: 830, dtype: object

In [ ]:
    def find_best_model(self):

        """

        Finds the best-performing regression model among a predefined list of models.

        Returns:
            tuple: A tuple containing the best model, its hyperparameters, and performance metrics.

        """

        # Define a list of models and their respective folders
        models_folders = [
            ("Linear Regression", "models/regression/linear_regression"),
            ("Decision Tree", "models/regression/decision_tree"),
            ("Random Forest", "models/regression/random_forest"),
            ("Gradient Boosting", "models/regression/gradient_boosting")
        ]

        best_model_name = None
        best_model = None
        best_hyperparameters = {}
        best_performance_metrics = {}
        best_rmse = float('inf')

        for model_name, folder in models_folders:
            # Load the model
            model_filename = os.path.join(folder, "model.joblib")
            loaded_model = joblib.load(model_filename)

            # Load hyperparameters
            hyperparameters_filename = os.path.join(folder, "hyperparameters.json")
            with open(hyperparameters_filename, "r") as hyperparameters_file:
                hyperparameters = json.load(hyperparameters_file)

            # Load performance metrics
            metrics_filename = os.path.join(folder, "metrics.json")
            with open(metrics_filename, "r") as metrics_file:
                performance_metrics = json.load(metrics_file)

            # Check if the current model has a lower RMSE
            if performance_metrics["validation_RMSE"] < best_rmse:
                best_rmse = performance_metrics["validation_RMSE"]
                best_model_name = model_name
                best_model = loaded_model
                best_hyperparameters = hyperparameters
                best_performance_metrics = performance_metrics

        print("Best Model:", best_model_name)
        print("Best Hyperparameters:", best_hyperparameters)
        print("Best Performance Metrics:", best_performance_metrics)

        return best_model, best_hyperparameters, best_performance_metrics

In [45]:
import json
import os
import joblib


def find_best_model(task_folder):

    """

    Finds the best-performing regression model among a predefined list of models.

    Returns:
        tuple: A tuple containing the best model, its hyperparameters, and performance metrics.

    """

    # Define a list of models and their respective folders
    models_folders = [
        ("Linear Regression", f"{task_folder}logistic_regression"),
        ("Decision Tree", f"{task_folder}decision_tree"),
        ("Random Forest", f"{task_folder}random_forest"),
        ("Gradient Boosting", f"{task_folder}gradient_boosting")
    ]

    best_model_name = None
    best_model = None
    best_hyperparameters = {}
    best_performance_metrics = {}
    best_mean = 0

    for model_name, folder in models_folders:
        # Load the model
        model_filename = os.path.join(folder, "model.joblib")
        loaded_model = joblib.load(model_filename)
        print(model_filename)

        # Load hyperparameters
        hyperparameters_filename = os.path.join(folder, "hyperparameters.json")
        with open(hyperparameters_filename, "r") as hyperparameters_file:
            hyperparameters = json.load(hyperparameters_file)

        # Load performance metrics
        metrics_filename = os.path.join(folder, "metrics.json")
        with open(metrics_filename, "r") as metrics_file:
            performance_metrics = json.load(metrics_file)

        # Check mean performance metric
        Total = performance_metrics['validation_accuracy']+performance_metrics['precision']+performance_metrics['recall']+performance_metrics['f1_score']
        mean = Total / 4
        print("mean =", mean)

        # Check if the current model has a lower RMSE
        if mean > best_mean:
            best_mean = mean
            best_model_name = model_name
            best_model = loaded_model
            best_hyperparameters = hyperparameters
            best_performance_metrics = performance_metrics

    print("Best Model:", best_model_name)
    print("Best Hyperparameters:", best_hyperparameters)
    print("Best Performance Metrics:", best_performance_metrics)
    print("best mean:", best_mean)

    return best_model, best_hyperparameters, best_performance_metrics

In [46]:
best_model, best_hyperparameters, best_performance_metrics = find_best_model("models/classification/")

models/classification/logistic_regression/model.joblib
mean = 0.3109622636975654
models/classification/decision_tree/model.joblib
mean = 0.3016705364719999
models/classification/random_forest/model.joblib
mean = 0.3856369118571704
models/classification/gradient_boosting/model.joblib
mean = 0.41596681871724783
Best Model: Gradient Boosting
Best Hyperparameters: {'learning_rate': 0.2, 'max_depth': 4, 'max_features': None, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 50}
Best Performance Metrics: {'validation_accuracy': 0.41379310344827586, 'precision': 0.42408731619257944, 'recall': 0.4124835203741749, 'f1_score': 0.41350333485396096}
best mean: 0.41596681871724783


In [52]:
data = pd.read_csv(r"AirbnbData/Raw_Data/tabular_data/listing.csv")

In [53]:
data = data.head(2)

In [54]:
data = data.to_dict()

In [55]:
data

{'Unnamed: 0': {0: 0, 1: 1},
 'ID': {0: 'f9dcbd09-32ac-41d9-a0b1-fdb2793378cf',
  1: '1b4736a7-e73e-45bc-a9b5-d3e7fcf652fd'},
 'Category': {0: 'Treehouses', 1: 'Treehouses'},
 'Title': {0: 'Red Kite Tree Tent - Ynys Affalon',
  1: 'Az Alom Cabin - Treehouse Tree to Nature Cabin'},
 'Description': {0: "<class 'str'>", 1: "<class 'str'>"},
 'Amenities': {0: "['What this place offers', 'Bathroom', 'Shampoo', 'Bedroom and laundry', 'Essentials', 'Towels, bed sheets, soap and toilet paper', 'Heating and cooling', 'Indoor fireplace', 'Heating', 'Home safety', 'Smoke alarm', 'Carbon monoxide alarm', 'Fire extinguisher', 'First aid kit', 'Kitchen and dining', 'Kitchen', 'Space where guests can cook their own meals', 'Cooking basics', 'Pots and pans, oil, salt and pepper', 'Location features', 'Private entrance', 'Separate street or building entrance', 'Parking and facilities', 'Free parking on premises', 'Services', 'Long-term stays allowed', 'Allow stays of 28 days or more', 'Not included', '

In [3]:
import pandas as pd
data=pd.read_csv('/Users/ryanhughes/Desktop/Aicore/Airbnb/Airbnb/AirbnbData/Processed_Data/clean_tabular_data/clean_tabular_data.csv')

In [6]:
data['bedrooms'].unique()

array([ 1,  2,  5,  3,  4,  8,  6, 10,  7])

In [7]:
def map_Bedrooms(data):
    unique_bedrooms = sorted(data['bedrooms'].unique())
    bedroom_mapping = {bedroom: idx for idx, bedroom in enumerate(unique_bedrooms)}

    # Apply the mapping
    data['bedrooms_mapped'] = data['bedrooms'].map(bedroom_mapping)

In [8]:
map_Bedrooms(data)

In [9]:
data

,Unnamed: 0,ID,Category,Title,Description,Amenities,Location,guests,beds,bathrooms,...,Cleanliness_rating,Accuracy_rating,Communication_rating,Location_rating,Check-in_rating,Value_rating,amenities_count,url,bedrooms,bedrooms_mapped
0,0,f9dcbd09-32ac-41d9-a0b1-fdb2793378cf,Treehouses,Red Kite Tree Tent - Ynys Affalon,<class 'str'>,"['What this place offers', 'Bathroom', 'Shampo...",Llandrindod Wells United Kingdom,2,1.0,1.0,...,4.6,4.7,4.3,5.0,4.3,4.3,13.0,https://www.airbnb.co.uk/rooms/26620994?adults...,1,0
1,1,1b4736a7-e73e-45bc-a9b5-d3e7fcf652fd,Treehouses,Az Alom Cabin - Treehouse Tree to Nature Cabin,<class 'str'>,"['What this place offers', 'Bedroom and laundr...",Guyonvelle Grand Est France,3,3.0,0.0,...,4.3,4.7,4.6,4.9,4.7,4.5,8.0,https://www.airbnb.co.uk/rooms/27055498?adults...,1,0
2,2,d577bc30-2222-4bef-a35e-a9825642aec4,Treehouses,Cabane Entre Les Pins\n🌲🏕️🌲,<class 'str'>,"['What this place offers', 'Scenic views', 'Ga...",Duclair Normandie France,4,2.0,1.5,...,4.2,4.6,4.8,4.8,4.8,4.7,51.0,https://www.airbnb.co.uk/rooms/51427108?adults...,1,0
3,3,ca9cbfd4-7798-4e8d-8c17-d5a64fba0abc,Treehouses,Tree Top Cabin with log burner & private hot tub,<class 'str'>,"['What this place offers', 'Bathroom', 'Hot wa...",Barmouth Wales United Kingdom,2,1.0,1.0,...,4.8,4.9,4.9,4.9,5.0,4.6,23.0,https://www.airbnb.co.uk/rooms/49543851?adults...,1,0
4,4,cfe479b9-c8f8-44af-9bc6-46ede9f14bb5,Treehouses,Treehouse near Paris Disney,<class 'str'>,"['What this place offers', 'Bathroom', 'Hair d...",Le Plessis-Feu-Aussoux Île-de-France France,4,3.0,1.0,...,5.0,4.9,5.0,4.7,5.0,4.7,32.0,https://www.airbnb.co.uk/rooms/935398?adults=1...,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
824,825,cfbc88da-e88c-415a-b397-108d0948c4ba,Beachfront,Lancing Beach Apartment,<class 'str'>,"['What this place offers', 'Bathroom', 'Hair d...",Lancing United Kingdom,4,2.0,1.5,...,4.9,5.0,5.0,5.0,4.9,4.8,33.0,https://www.airbnb.co.uk/rooms/12680472?adults...,2,1
825,826,4fea5054-f999-4c07-addc-67e4d893deab,Beachfront,Apartment,<class 'str'>,"['What this place offers', 'Bathroom', 'Hair d...",Brighton and Hove England United Kingdom,2,1.0,1.0,...,4.8,5.0,4.9,4.9,5.0,4.9,54.0,https://www.airbnb.co.uk/rooms/48565992?adults...,1,0
826,827,282118e2-049e-4d9f-b2f2-b47477881b07,Beachfront,Sea front flat with a stunning view!,<class 'str'>,"['What this place offers', 'Scenic views', 'Be...",East Sussex England United Kingdom,4,2.0,1.5,...,4.8,5.0,5.0,5.0,5.0,4.8,38.0,https://www.airbnb.co.uk/rooms/49742544?adults...,2,1
827,828,9ebf9cec-624e-480e-8704-dffa7cb1fe51,Beachfront,MP713 - Camber Sands Holiday Park - Sleeps 6 +...,<class 'str'>,"['What this place offers', 'Bathroom', 'Hot wa...",Camber England United Kingdom,6,3.0,2.0,...,4.7,4.8,5.0,5.0,5.0,4.7,24.0,https://www.airbnb.co.uk/rooms/47777462?adults...,2,1
